## **1. Explore the data**

In [9]:
# Prepare my environment
import pandas as pd
import sqlite3
df = pd.read_csv('/Users/viridianachow/Desktop/Projects/Exploring NYC Public School Test Result Scores/schools.csv')
cnn = sqlite3.connect('schools.db')
df.to_sql('schools', cnn)
%load_ext sql
%sql sqlite:///schools.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @schools.db'

In [11]:
%%sql

-- select all columns but only 10 rows

SELECT *
FROM schools
LIMIT 10;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


index,school_name,borough,building_code,average_math,average_reading,average_writing,percent_tested
0,"New Explorations into Science, Technology and Math High School",Manhattan,M022,657,601,601,None
1,Essex Street Academy,Manhattan,M445,395,411,387,78.9
2,Lower Manhattan Arts Academy,Manhattan,M445,418,428,415,65.1
3,High School for Dual Language and Asian Studies,Manhattan,M445,613,453,463,95.9
4,Henry Street School for International Studies,Manhattan,M056,410,406,381,59.7
5,Bard High School Early College,Manhattan,M097,634,641,639,70.8
6,Urban Assembly Academy of Government and Law,Manhattan,M445,389,395,381,80.8
7,Marta Valle High School,Manhattan,M025,438,413,394,35.6
8,University Neighborhood High School,Manhattan,M446,437,355,352,69.9
9,New Design High School,Manhattan,M445,381,396,372,73.7


## **2. Finding missing values**
It looks like the first school in our database had no data in the percent_tested column!

Let's identify how many schools have missing data for this column, indicating schools that did not report the percentage of students tested.

To understand whether this missing data problem is widespread in New York, we will also calculate the total number of schools in the database.

In [22]:
%%sql 

-- find schools with percent_tested as null
-- find the total number of schools in the database
-- find the percentage of percent_tested as null against the total number of schools

SELECT
    COUNT(*) - COUNT(percent_tested) AS num_testing_null,
    COUNT(*) AS num_total_schools,
    ((COUNT(*) - COUNT(percent_tested)) * 100 / COUNT(*)) AS percent_num_testing_null
FROM schools;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


num_testing_null,num_total_schools,percent_num_testing_null
20,375,5


## **3. Schools by building code**
There are 20 schools with missing data for percent_tested, which only makes up 5% of all rows in the database.

Now let's turn our attention to how many schools there are. When we displayed the first ten rows of the database, several had the same value in the building_code column, suggesting there are multiple schools based in the same location. Let's find out how many unique school locations exist in our database.

In [26]:
%%sql

-- count unique building_code value

SELECT 
    COUNT(DISTINCT building_code) AS num_building_code,
    COUNT(DISTINCT building_code) * 100 / COUNT(*) AS percent_unique_building_code
FROM schools;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


num_building_code,percent_unique_building_code
233,62


## **4. Best schools for math**
Out of 375 schools, only 233 (62%) have a unique building_code!

Now let's start our analysis of school performance. As each school reports individually, we will treat them this way rather than grouping them by building_code.

First, let's find all schools with an average math score of at least 80% (out of 800).

In [29]:
%%sql

-- find out schools with average_math with at least 80% test score

SELECT 
    school_name,
    average_math
FROM schools
WHERE average_math >= 0.8 * 800
ORDER BY average_math DESC;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


school_name,average_math
Stuyvesant High School,754
Bronx High School of Science,714
Staten Island Technical High School,711
Queens High School for the Sciences at York College,701
"High School for Mathematics, Science, and Engineering at City College",683
Brooklyn Technical High School,682
Townsend Harris High School,680
High School of American Studies at Lehman College,669
"New Explorations into Science, Technology and Math High School",657
Eleanor Roosevelt High School,641


## **5. Lowest reading score**
Wow, there are only ten public schools in New York City with an average math score of at least 640!

Now let's look at the other end of the spectrum and find the single lowest score for reading. We will only select the score, not the school, to avoid naming and shaming!

In [31]:
%%sql

-- find out the school with lowest average_reading score

SELECT 
    MIN(average_reading) AS lowest_avg_reading,
    MIN(average_reading) * 100 / 800 AS percentage_of_total_score
FROM schools;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


lowest_avg_reading,percentage_of_total_score
302,37


## **6. Best writing school**
The lowest average score for reading across schools in New York City is less than 40% of the total available points!

Now let's find the school with the highest average writing score.

In [32]:
%%sql

-- find the school with highest average_writing

SELECT 
    school_name,
    MAX(average_writing) AS highest_avg_writing
FROM schools;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


school_name,highest_avg_writing
Stuyvesant High School,693


## **7. Top 10 schools**
An average writing score of 693 is pretty impressive!

This top writing score was at the same school that got the top math score, Stuyvesant High School. Stuyvesant is widely known as a perennial top school in New York.

What other schools are also excellent across the board? Let's look at scores across reading, writing, and math to find out.

In [39]:
%%sql

-- find the schools with top average total SAT score

SELECT
    school_name,
    average_math + average_writing + average_reading AS avg_total_score
FROM schools
ORDER BY avg_total_score DESC
LIMIT 10;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


school_name,avg_total_score
Stuyvesant High School,2144
Staten Island Technical High School,2041
Bronx High School of Science,2041
High School of American Studies at Lehman College,2013
Townsend Harris High School,1981
Queens High School for the Sciences at York College,1947
Bard High School Early College,1914
Brooklyn Technical High School,1896
Eleanor Roosevelt High School,1889
"High School for Mathematics, Science, and Engineering at City College",1889


## **8. Ranking boroughs**
There are four schools with average SAT scores of over 2000! Now let's analyze performance by New York City borough.

We will build a query that calculates the number of schools and the average SAT score per borough!

In [40]:
%%sql

-- find the number of schools and average SAT score by borough

SELECT
    borough,
    COUNT(*) AS num_schools,
    SUM(average_math + average_writing + average_reading) / COUNT(*) AS avg_total_score
FROM schools
GROUP BY borough
ORDER BY avg_total_score DESC;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


borough,num_schools,avg_total_score
Staten Island,10,1439
Queens,69,1345
Manhattan,89,1340
Brooklyn,109,1230
Bronx,98,1202


## **9. Brooklyn numbers**
It appears that schools in Staten Island, on average, produce higher scores across all three categories. However, there are only 10 schools in Staten Island, compared to an average of 91 schools in the other four boroughs!

For our final query of the database, let's focus on Brooklyn, which has 109 schools. We wish to find the top five schools for math performance.

In [41]:
%%sql

-- find the top 5 Brooklyn schools with highest average_math

SELECT
    school_name,
    average_math
FROM schools
WHERE borough = 'Brooklyn'
ORDER BY average_math DESC
LIMIT 5;

 * sqlite:///schools.db
   sqlite:///schoools.db
Done.


school_name,average_math
Brooklyn Technical High School,682
Brooklyn Latin School,625
Leon M. Goldstein High School for the Sciences,563
Millennium Brooklyn High School,553
Midwood High School,550
